In [ ]:
import spacy
import pandas as pd
import numpy as np
import json
import pickle
from collections import OrderedDict
import re
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
family = ['father','mother','aunt','wife','daughter','sibling','twin','family','heir','ancestor',
          'brother','uncle','sister','niece','grand','cousin','adopt','relat','nephew','son','child','divorce']
friend = ['friend','playmate','widow','frien']
romance = ['ex','lover','love','girlfriend','attraction','boyfriend','affair','engage',
          'fiance','crush','date','sweet','partner','couple','flirt','marr']
enemy = ['enem','victim','traitor','compet','parties','riv','dislike','foe','death','counter',
         'murder','accuse','duel','conflicts','hate','foil','opposition','disguise','kill']
acquaintance = ['acqua','coworker','student','prof','pup','roommate','school','work','host','housemates','companion',
               'neighbor','roomate','wizard','ally','allies','flatmate','mate','group','miss','member','peasant',
               'coll','train','comrade','land']
service = ['fellow','assist','doctor','detect','devil','master','mistress','slave','rule','henchman',
           'employer','serv','lead','law','king','prison',',coach','proph','resear','edit','ward',
           'cook','sale','officer','boss','office','lord','emperor','interview','chief','support','advis',
          'nurse','man','owner','mentor','benef','manager','ruler','starbuck','super','tetrarch','tour',
          'counsel','judge','merchant','employ','flower','general','warder','house','soldier','maid','major','help',
           'patient','cook','ward','business','bank','tenant','keeper','captain','tutor','keeper','actor','buy','lend',
          'porter','caller','scout','hire','protect','guide','attorney','coach','caretaker','associate','advers',
          'interrogator','harpooner','henchman']
label_dict = OrderedDict([('enemy',enemy),('romance',romance),('family',family),('friend',friend),
                         ('acquaintance',acquaintance),('service',service)])

def clean_label(label):
    for key,value in label_dict.iteritems():
        for v in value:
            if re.search(v,label):
                return key
    return 'ambiguous'

def clean_all(label_dict):
    for key in label_dict:
        for pair in label_dict[key]:
            label_dict[key][pair] = clean_label(label_dict[key][pair])
    return label_dict

In [ ]:
def unpack(lst):
    return ' '.join([i[0] for i in lst]), [i[1] for i in lst]

def parse_pair_name(pair):
        pair_parse = [probablepeople.tag(i)[0] for i in pair]
        pair_parse = [(i['GivenName'].strip() if 'GivenName' in i else '',
                      i['Surname'].strip() if 'Surname' in i else '') for i in pair_parse]
        return pair_parse

def match_pairs_one(sentences,labels):
    total = 0
    success = 0
    for pair in sentences:
        total += 1
        pair_tmp = (i.lower().strip() for i in pair)
        relation = []
        # 1. exact match
        for label_pair in labels:
            if pair_tmp == label_pair:
                success += 1
                sentences[pair] = [sentence[pair],labels[label_pair]]
                break
        # 2. match first and last name
        try:
            pair_parse = parse_pair_name(pair_tmp)
            for label_pair in labels:
                label_pair_parse = parse_pair_name(label_pair)
                if label_pair_parse == pair_parse or label_pair_parse[0] == pair_parse[0] or label_pair_parse[1] == pair_parse[1]:
#                     print "Matched: {} == {}".format(pair_parse,label_pair_parse)
                    success += 1
                    relation.append(labels[label_pair])
                    break
        except:
            pass
        if not relation:
#             print "Unmatched: {} != {}".format(pair_parse,label_pair_parse)

#         sentences[pair] = list(unpack(sentences[pair])) + [relation if relation else ['ambiguous']]

    print 'Total:{}, Match:{}'.format(total,success)
#     return sentences

def match_pairs_all(sentences_all,labels_all):
    new_dict = {}
    for i in sentences_all:
        if i in labels_all:
            new_dict[i] = match_pairs_one(sentences_all[i],labels_all[i])
    return new_dict
